In [2]:
from RCA import RCA
from iGA import Improved_GA
import numpy as np
from numpy import sqrt
from scipy import special

In [3]:
def Q(EsNo):
    BER=1/2*(1-special.erf(sqrt(EsNo)))
    return BER

In [ ]:
#RCA
N=1024
K=512
M_list=[4,16,256]
for M in M_list:
    EsNodB=np.arange(0,30,0.5)
    BLER=np.zeros(len(EsNodB))
    for i,SNR in enumerate(EsNodB):
        #gamma == EsNo(True value)
        const=RCA()
        gamma=np.sort(np.exp(const.main_const(N,K,SNR,M,soft_output=True)))[::-1]
        gamma=gamma[:K]
        #print(gamma.shape)
        BLER[i]=1-np.prod((1-Q(gamma)))
        if BLER[i]==0:
            BLER=BLER[:i]
            break

    filename="EST_RCA_"+str(N)+"_"+str(K)+"_"+str(M)
    with open(filename,'w') as f:
        print("#EsNodB,BLER,BER",file=f)
        
        for i in range(len(BLER)):
            #print(EsNodB[i])
            print(str(EsNodB[i]),str(BLER[i]),file=f)

In [30]:
N=1024
K=512
M=16
SNR=12
const=Improved_GA()
gamma=(const.main_const(N,K,SNR,M,soft_output=True))[::-1]
#print(gamma)
gamma=np.exp(gamma[:K])
#print(gamma)
print(1-np.prod((1-Q(gamma/4))))

0.05781393485731501


/tmp/ipykernel_2377250/2626261011.py:8: RuntimeWarning: overflow encountered in exp
  gamma=np.exp(gamma[:K])


In [ ]:
#iGA
N=1024
K=512
M_list=[4,16,256]
for M in M_list:
    EsNodB=np.arange(0,30,0.5)
    BLER=np.zeros(len(EsNodB))
    for i,SNR in enumerate(EsNodB):
        #gamma == EsNo(True value)
        const=Improved_GA()
        gamma=np.sort((const.main_const(N,K,SNR,M,soft_output=True)))[::-1]
        gamma=gamma[:K]
        print(gamma)
        BLER[i]=1-np.prod((1-Q(gamma)))
        if BLER[i]==0:
            BLER=BLER[:i]
            break

    filename="EST_iGA_"+str(N)+"_"+str(K)+"_"+str(M)
    with open(filename,'w') as f:
        print("#EsNodB,BLER,BER",file=f)
        
        for i in range(len(BLER)):
            #print(EsNodB[i])
            print(str(EsNodB[i]),str(BLER[i]),file=f)
